In [1]:
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from slsim.Pipelines.skypy_pipeline import SkyPyPipeline
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
import numpy as np
from slsim.selection import deflector_cut
import corner
from lenstronomy.Util import constants
from colossus.cosmology import cosmology
import numpy.random as random
from slsim.Util import param_util
import pandas as pd
import sys
from slsim.lens import Lens
from slsim.Deflectors.deflector import Deflector
# sys.path.append('../slsim')
# from lens_pop import LensPop
# from lensed_population_base import LensedPopulationBase
# sys.path.append('../slsim/Deflectors')  # 別ディレクトリのパスを追加
# from deflectors_base import DeflectorsBase
# from compound_lens_halos_galaxies import CompoundLensHalosGalaxies
# from lenstronomy.Util import constants
# from velocity_dispersion import vel_disp_composite_model
# sys.path.append('../slsim/Pipelines')  # 別ディレクトリのパスを追加
# from sl_hammocks_pipeline import SLHammocksPipeline


# Galaxy-galaxy simulations

This notebook walks through the basics of simulating a galaxy-galaxy strong lensing population.
The underlying 

assumptions of the galaxy populations (for both lenses and sources) are drawn from a population pre-configured

 and rendered through [SkyPy](https://github.com/skypyproject/skypy). The specific settings are described in the [readme file](https://github.com/LSST-strong-lensing/slsim/tree/gg-lensing/data/SkyPy).

The notebook goes in three steps:

1. The populations of lenses and sources is produced.
2. Random draws of the population are generated and realized as images
3. The full population is generated in catalogue form
4. the full population is represented in a corner plot



## Generate population of galaxies and (potential) deflectors
The LensPop() class in the slsim package is used to produce a set of galaxies (as lenses and sources)

as seen on the sky within a certain sky area.
We use the default SkyPy configuration file. Alternative configuration

files can be used.

In [2]:
# define a cosmology

cosmo_col = cosmology.setCosmology('planck18')
cosmo = FlatLambdaCDM(H0=cosmo_col.H0, Om0=cosmo_col.Om0)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml'
# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop2 = LensPop(
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    sky_area=sky_area,
    cosmo=cosmo,
)

In [6]:
sys.path.append('../slsim/Pipelines')
import solve_lenseq
import lens_gals
# import lens_halo
# import global_value as g

# define a cosmology

cosmo_col = cosmology.setCosmology('planck18')
cosmo = FlatLambdaCDM(H0=cosmo_col.H0, Om0=cosmo_col.Om0)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = None
# {
#     'sig_c': 0.33,
#     'sig_c_sh': 0.33,
#     'sig_mcen': 0.2,
#     'sig_msat': 0.2,
#     'sig_tb': 0.46,
#     'TYPE_GAL_SIZE': 'vdW23',
#     'frac_SM_IMF': 1.715,
#     'TYPE_SMHM': 'true',
#     'area': sky_area.value,
#     'zmin': 0.01,
#     'zmax': 5.0,
#     'Mhmin': 1e11,
#     'Mhmax': 1e16,
#     'switch_sub': False,
#     'halo_gal_pop_array': np.empty((0, 10), float),
#     'dz': 0.001,
#     'zz_ar': np.arange(0.01, 5.001, 0.001),
#     'dlogMh': 0.001,
#     'dlnMh': np.log(10**0.001),
#     'MMh': 10**np.arange(np.log10(1e11), np.log10(1e16), 0.001),
#     'min_Msh': 1e11 / 10
# }
#{"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim_KTA/data/SkyPy/lsst-like.yml'
slhammocks_config='/Users/ktabe/Research/strong-lens-SNe/slsim_KTA/data/SL-Hammocks/gal_pop_Salpeter_10deg2_zl2.csv'
# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_type="halo-models",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    slhammocks_config=slhammocks_config,
    sky_area=sky_area,
    cosmo=cosmo,
)

In [3]:
kwargs_deflector_cut = {"z_min": 0.01, "z_max": 2.0}

# skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim_KTA/data/SL-Hammocks/gal_pop_Salpeter_10deg2_zl2.csv'
# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop3 = LensPop(
    deflector_type="halo-models",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    slhammocks_config=None,
    sky_area=sky_area,
    cosmo=cosmo,
)

In [14]:
np.log10(min(gg_lens_pop3._lens_galaxies._galaxy_select['halo_mass'])), max(gg_lens_pop._lens_galaxies._galaxy_select['z'])

(11.0, 2.0)

In [9]:
gg_lens_pop._lens_galaxies._galaxy_select

,z,halo_mass,halo_mass_acc,e_h,p_h,concentration,stellar_mass,e_g,p_g,tb,vel_disp,mag_g,mag_r,mag_i,mag_z,mag_Y,e1_light,e2_light,e1_mass,e2_mass
0,0.057,1.235947e+11,0.0,0.149634,144.400719,13.624457,7.625149e+08,0.109018,116.899515,1.021455,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.057,1.235947e+11,0.0,0.149634,144.400719,13.624457,7.625149e+08,0.109018,116.899515,1.021455,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0.079,2.285599e+11,0.0,0.234507,134.565679,12.061913,4.462336e+09,0.355826,141.257081,1.069006,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0.079,2.285599e+11,0.0,0.234507,134.565679,12.061913,4.462336e+09,0.355826,141.257081,1.069006,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,0.081,2.328091e+11,0.0,0.324689,-64.653968,6.251633,3.629105e+09,0.341456,6.445627,0.501659,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65333,4.999,1.086426e+11,0.0,0.283190,52.199633,2.057132,6.225776e+08,0.327147,72.683730,0.043704,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
65334,4.999,1.603245e+11,0.0,0.151613,-147.080301,3.044508,1.016017e+09,0.461124,-156.931136,0.177810,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
65335,4.999,1.698244e+11,0.0,0.237754,-31.985798,2.984187,1.048405e+09,0.561852,-74.427449,0.125641,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
65336,5.000,1.180321e+11,0.0,0.275803,-18.939015,3.873024,5.126302e+08,0.321513,-3.241769,0.094848,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [3]:
kwargs_deflector_base = {
    'sig_c': 0.33,
    'sig_c_sh': 0.33,
    'sig_mcen': 0.2,
    'sig_msat': 0.2,
    'sig_tb': 0.46,
    'TYPE_GAL_SIZE': 'vdW23',
    'frac_SM_IMF': 1.715,
    'TYPE_SMHM': 'true',
    'zmin': 0.01,
    'zmax': 5.0,
    'Mhmin': 1e11,
    'Mhmax': 1e16,
    'switch_sub': False,
    'halo_gal_pop_array': np.empty((0, 10), float),
    'dz': 0.001,
    'zz_ar': np.arange(0.01, 5.001, 0.001),
    'dlogMh': 0.001,
    'dlnMh': np.log(10**0.001),
    'MMh': 10**np.arange(np.log10(1e11), np.log10(1e16), 0.001),
    'min_Msh': 1e11 / 10
}

kwargs_deflector_cut = {
    'sig_c': 0.10,
    'TYPE_GAL_SIZE': 'OT20',
    'frac_SM_IMF': 1.715,
    'TYPE_SMHM': 'true',
    'zmin': 0.01,
    'zmax': 3.0,
}

def update_dictionary(original_dict, **kwargs):
    for key, value in kwargs.items():
        if key in original_dict:
            original_dict[key] = value
        else:
            print(f"Key '{key}' not found in the dictionary")
    return original_dict

udict = update_dictionary(kwargs_deflector_base, **kwargs_deflector_cut)
udict

{'sig_c': 0.1,
 'sig_c_sh': 0.33,
 'sig_mcen': 0.2,
 'sig_msat': 0.2,
 'sig_tb': 0.46,
 'TYPE_GAL_SIZE': 'OT20',
 'frac_SM_IMF': 1.715,
 'TYPE_SMHM': 'true',
 'zmin': 0.01,
 'zmax': 3.0,
 'Mhmin': 100000000000.0,
 'Mhmax': 1e+16,
 'switch_sub': False,
 'halo_gal_pop_array': array([], shape=(0, 10), dtype=float64),
 'dz': 0.001,
 'zz_ar': array([0.01 , 0.011, 0.012, ..., 4.998, 4.999, 5.   ]),
 'dlogMh': 0.001,
 'dlnMh': 0.0023025850929939925,
 'MMh': array([1.00000000e+11, 1.00230524e+11, 1.00461579e+11, ...,
        9.93116048e+15, 9.95405417e+15, 9.97700064e+15]),
 'min_Msh': 10000000000.0}

In [66]:
from slsim.Deflectors.DeflectorTypes.nfw_hernquist import NFWHernquist
source_dict = gg_lens_pop._sources.draw_source()
deflector_dict = gg_lens_pop._lens_galaxies.draw_deflector()
deflector_dict2 = gg_lens_pop2._lens_galaxies.draw_deflector()
source_dict, deflector_dict, deflector_dict2

(<Row index=56672>
         z                  M                            coeff                       ellipticity       physical_size      stellar_mass       angular_size         mag_g              mag_r             mag_i              mag_z              mag_Y                e1                 e2         n_sersic
                                                                                                            kpc                                  rad                                                                                                                                                    
      float64            float64                       float64[5]                      float64            float64           float64            float64           float64            float64           float64            float64            float64            float64            float64       float64 
 ----------------- ------------------- ------------------------------------------ --------

In [9]:
deflector = NFWHernquist(deflector_dict=deflector_dict)
deflector.stellar_mass, deflector.deflector_center

(2502572000.0, array([-0.02647809, -0.05136608]))

In [64]:
gg_lens = Lens(
        source_dict,
        deflector_dict,
        cosmo,
        deflector_type="NFW_HERNQUIST",
        source_type="extended",
        lens_equation_solver="lenstronomy_analytical",
        variability_model=None,
        kwargs_variability=None,
        sn_type=None,
        sn_absolute_mag_band=None,
        sn_absolute_zpsys=None,
        test_area=4 * np.pi,
        mixgauss_means=None,
        mixgauss_stds=None,
        mixgauss_weights=None,
        magnification_limit=0.01,
        light_profile="single_sersic",
        lightcurve_time=None,
    )

gg_lens2 = Lens(
        source_dict,
        deflector_dict2,
        cosmo,
        deflector_type="EPL",
        source_type="extended",
        lens_equation_solver="lenstronomy_analytical",
        variability_model=None,
        kwargs_variability=None,
        sn_type=None,
        sn_absolute_mag_band=None,
        sn_absolute_zpsys=None,
        test_area=4 * np.pi,
        mixgauss_means=None,
        mixgauss_stds=None,
        mixgauss_weights=None,
        magnification_limit=0.01,
        light_profile="single_sersic",
        lightcurve_time=None,
    )

In [73]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.Analysis.lens_profile import LensProfileAnalysis
lens_model_list, kwargs_lens = gg_lens.deflector_mass_model_lenstronomy()
lens_model = LensModel(lens_model_list=lens_model_list, z_source_convention=1000)
lens_analysis = LensProfileAnalysis(lens_model=lens_model)
theta_E = lens_analysis.effective_einstein_radius(
    kwargs_lens, r_min=1e-3, r_max=1.0, num_points=50)
kwargs_lens,lens_model_list,theta_E

lens_model_list2, kwargs_lens2 = gg_lens2.deflector_mass_model_lenstronomy()
lens_model2 = LensModel(lens_model_list=lens_model_list2)
lens_analysis2 = LensProfileAnalysis(lens_model=lens_model2)
theta_E2 = lens_analysis2.effective_einstein_radius(
    kwargs_lens2, r_min=1e-3, r_max=1.0, num_points=50)
kwargs_lens,lens_model_list,theta_E,kwargs_lens2,lens_model_list2,theta_E2

([{'alpha_Rs': 1.391982291487466,
   'Rs': 11.441899605897744,
   'e1': -0.026877782716078196,
   'e2': 0.06876297301764561,
   'center_x': -0.009832829685977146,
   'center_y': 0.061441451688243824},
  {'Rs': 0.6916090744101633,
   'sigma0': 0.3080803233627541,
   'e1': -0.0970121841931694,
   'e2': 0.1732102242242809,
   'center_x': -0.009832829685977146,
   'center_y': 0.061441451688243824},
  {'gamma1': 0.01675355877613104,
   'gamma2': -0.06122245364205418,
   'ra_0': 0,
   'dec_0': 0},
  {'kappa': 0.05699482315612117, 'ra_0': 0, 'dec_0': 0}],
 ['NFW_ELLIPSE_CSE', 'HERNQUIST_ELLIPSE_CSE', 'SHEAR', 'CONVERGENCE'],
 0.34454409531650976,
 [{'theta_E': 0.16819887684638882,
   'gamma': 2,
   'e1': 0.0014557701980333943,
   'e2': -0.006813914890781982,
   'center_x': -0.0743951238937912,
   'center_y': -0.23741370803816797},
  {'gamma1': -0.14396362540265492,
   'gamma2': 0.11275161737464737,
   'ra_0': 0,
   'dec_0': 0},
  {'kappa': -0.008045392174212459, 'ra_0': 0, 'dec_0': 0}],
 ['EP

In [60]:
from lenstronomy.Util import util
from lenstronomy.Util import mask_util as mask_util
import lenstronomy.Util.multi_gauss_expansion as mge
from lenstronomy.Util import analysis_util
import scipy
r_min=1e-6
r_max=1.0
num_points=100
r_array = np.logspace(np.log10(r_min), np.log10(r_max), num_points)
center_x, center_y = analysis_util.profile_center(
    kwargs_lens, center_x=0, center_y=0
)
kappa_list = []
for r in r_array:
    x, y = util.points_on_circle(r, num_points=20)
    f_r = lens_model.kappa(
        x + center_x, y + center_y, kwargs=kwargs_lens, k=None
    )
    kappa_list.append(np.average(f_r))

kappa_r = kappa_list
# here we make a finer grid interpolation in log-log space
k_interp = scipy.interpolate.interp1d(np.log10(r_array), np.log10(kappa_r))
r_array = np.logspace(np.log10(r_min), np.log10(r_max), num_points * 10)
kappa_r = 10 ** k_interp(np.log10(r_array))

# we perform the integral in logarithmic steps of the convergence
kappa_r = np.array(kappa_r)
kappa_r_ = (kappa_r[1:] + kappa_r[:-1]) / 2
r_array_ = (r_array[1:] + r_array[:-1]) / 2
dlog_r = (np.log10(r_array[2]) - np.log10(r_array[1])) * np.log(10)
# add the mass within the innermost bin and assume it's constant
kappa_innermost = kappa_r[0] * np.pi * r_array[0] ** 2

# the first part is the logarithmic integrand, the second part the circle integrand
kappa_slice = kappa_r_ * dlog_r * r_array_ * (2 * np.pi * r_array_)
kappa_slice = np.append(kappa_innermost, kappa_slice)

kappa_cdf = np.cumsum(kappa_slice)
# calculate average convergence at radius
kappa_average = kappa_cdf / (np.pi * r_array**2)

# we interpolate as the inverse function and evaluate this function for average kappa = 1
# (assumes monotonic decline in average convergence)
# inv_interp = scipy.interpolate.interp1d(
#     np.log10(kappa_average), np.log10(r_array)
# )
# np.log10(kappa_average),inv_interp(max(np.log10(kappa_average)))
kappa_r
# try:
#     theta_e = 10 ** inv_interp(0)
# except:
#     theta_e = np.nan
# theta_e


array([0.55299998, 0.55299998, 0.55299998, 0.55299998, 0.55299998,
       0.55299997, 0.55299997, 0.55299997, 0.55299997, 0.55299997,
       0.55299996, 0.55299996, 0.55299996, 0.55299996, 0.55299995,
       0.55299995, 0.55299995, 0.55299995, 0.55299994, 0.55299994,
       0.55299994, 0.55299994, 0.55299993, 0.55299993, 0.55299993,
       0.55299993, 0.55299992, 0.55299992, 0.55299992, 0.55299992,
       0.55299991, 0.55299991, 0.55299991, 0.5529999 , 0.5529999 ,
       0.5529999 , 0.5529999 , 0.55299989, 0.55299989, 0.55299989,
       0.55299988, 0.55299988, 0.55299988, 0.55299987, 0.55299987,
       0.55299987, 0.55299986, 0.55299986, 0.55299986, 0.55299985,
       0.55299985, 0.55299984, 0.55299984, 0.55299984, 0.55299983,
       0.55299983, 0.55299982, 0.55299982, 0.55299982, 0.55299981,
       0.55299981, 0.5529998 , 0.5529998 , 0.55299979, 0.55299979,
       0.55299979, 0.55299978, 0.55299978, 0.55299977, 0.55299977,
       0.55299976, 0.55299976, 0.55299975, 0.55299975, 0.55299

# Test generate dataframe

In [82]:
g.sig_c

0.33

In [92]:

# Scatter of concentration params for halos & gals in log-normal distribution
sig_c = 0.33  # Intrinsic scatter (https://arxiv.org/abs/astro-ph/0608157)
sig_c_sh = 0.33  # Intrinsic scatter (https://arxiv.org/abs/astro-ph/0608157)
sig_mcen = 0.2  # Intrinsic scatter (https://iopscience.iop.org/article/10.3847/1538-4357/ac4cb4/pdf)
sig_msat = 0.2
sig_tb = 0.46

# method to calculate galaxy half-light effective size (or rb in Hernquist profile)
# options: 'vdW23'(JWST-base), 'oguri20'(simple), 'karmakar23'(IllustrisTNG-base), (default 'vdW23')
TYPE_GAL_SIZE = 'vdW23'

# fraction of Stellar mass-to-light ratio in respect to Chabrier IMF
# Chabrier: frac_SM_IMF=1.0, Salpeter: =1.715 (default 1.715)
frac_SM_IMF = 1.715

# type of fitting formula of the stellar-mass-halo-mass relation
# options: 'true','true_all','obs', see Berhoozi et al. 2019 Table J1
# default 'true'
TYPE_SMHM = 'true'
paramc, params = lens_gals.gals_init(TYPE_SMHM)

# cosmological parameters
area = sky_area.value
zmin=0.01
zmax=5.0
Mhmin=1e11
Mhmax=1e16
switch_sub=False
halo_gal_pop_array = np.array([])
halo_gal_pop_array = np.empty((0,10), float)
dz = 0.001
zz_ar = np.arange(zmin, zmax + dz, dz)
dlogMh = 0.001
dlnMh = np.log(10**dlogMh)
MMh = 10**np.arange(np.log10(Mhmin),
                    np.log10(Mhmax), dlogMh)
min_Msh =Mhmin/10.

frac_SM_IMF = 1.715
#     if switch_sub:
#         n_bins = 100
#         output_length = n_bins-1
#         interp_dnsh, interp_msh_acc_Mh = lens_subhalo.create_interp_dndmsh(0, zmax+0.1, Mhmin/2., Mhmax*2., min_Msh, cosmo, n_bins=n_bins)

# Start for loop-1 of redshift
for zz in zz_ar:
    zz2 = np.full(len(MMh), zz)
    NNh = area * \
        lens_halo.dNhalodzdlnM_lens(MMh, zz2, cosmo_col) * dlnMh * dz
    Nh = np.random.poisson(NNh)
    indices = np.nonzero(Nh)[0]
    if len(indices) == 0:
        continue

    zl_tab = np.repeat(zz2[indices], Nh[indices])
    Mhosthl_tab  = np.repeat(MMh[indices], Nh[indices])
    conhl_tab = lens_halo.concent_m_w_scatter(Mhosthl_tab, zz, g.sig_c)
    # in physical [Mpc/h]
    eliphl_tab, polarhl_tab = solve_lenseq.gene_e_ang_halo(Mhosthl_tab)

    mshsat_tot = 0
    Mhosthl_tab_re = Mhosthl_tab

#         if switch_sub == True:
#             Mhl_zl_tab_vec = np.vstack((np.log10(Mhosthl_tab),  zl_tab)).T
#             dnshp = interp_dnsh(Mhl_zl_tab_vec).reshape(
#                 len(Mhosthl_tab), output_length)
#             msh_acc_Mh = interp_msh_acc_Mh(Mhl_zl_tab_vec).reshape(
#                 len(Mhosthl_tab), output_length)
#             dnsh = np.where((msh_acc_Mh > 0.5) | (dnshp < 1.0e-4), 0., dnshp)
#             mmsh_acc = msh_acc_Mh*Mhosthl_tab.reshape(len(Mhosthl_tab), -1)
#             mmsh = np.logspace(np.log10(min_Msh), np.log10(
#                 Mhosthl_tab/2.), n_bins).T[:, 1:]  # in [Modot/h]
#             NNsh = np.random.poisson(dnsh)

#             # Start: for-loop-2 of host halos
#             for j, (mh, zl) in enumerate(zip(Mhosthl_tab, zl_tab)):
#                 indices_sh = np.nonzero(NNsh[j])[0]
#                 cut_Nsh = NNsh[j][indices_sh]
#                 cut_msh = mmsh[j][indices_sh]
#                 cut_msh_acc = mmsh_acc[j][indices_sh]
#                 # Subhalo mass
#                 msh_tab = np.repeat(cut_msh, cut_Nsh)
#                 msh_acc_tab = np.repeat(cut_msh_acc, cut_Nsh)
#                 zsub_tab = np.full(len(msh_tab), zz)
#                 mshsat_tot = 0
#                 # Start: If at least one subhalo exists in loop-2 of host halos
#                 if len(msh_tab) != 0:
#                     # Satellite gals
#                     msat_ave = lens_gals.stellarmass_halomass(
#                         msh_acc_tab/(cosmo.H0/100.), zl, g.params, g.frac_SM_IMF)*(cosmo.H0/100.)  # KA
#                     msat_scat = np.random.lognormal(0.0, g.sig_msat, len(msh_acc_tab))
#                     msat_tab = msat_ave*msat_scat

#                     mshsat_tot = sum(msh_tab)+sum(msat_tab)
#                     elipsh_tab, polarsh_tab = solve_lenseq.gene_e_ang_halo(msh_acc_tab)
#                     con_sh_ave = np.where(lens_subhalo.concent_m_sub_ando(msh_tab, zl, cosmo) > lens_halo.concent_m(msh_tab, zl),
#                                       lens_subhalo.concent_m_sub_ando(msh_tab, zl, cosmo), lens_halo.concent_m(msh_tab, zl))
#                     cor_con_sh = mass_so.M_to_R(
#                         msh_acc_tab, zl, 'vir') / mass_so.M_to_R(msh_tab, zl, 'vir')
#                     con_sh_tab = con_sh_ave * \
#                         cor_con_sh * np.random.lognormal(0.0, g.sig_c_sh, len(msh_tab))
#                     elipsat_tab, polarsat_tab = lens_gals.set_gals_param(polarsh_tab)
#                     tb_sat_tab = lens_gals.galaxy_size(
#                          msh_acc_tab, msat_tab/g.frac_SM_IMF, zz, cosmo, model=g.TYPE_GAL_SIZE, scatter=True, sig_tb=g.sig_tb)
#                     halogal_par_mat = np.hstack((zsub_tab.reshape(-1, 1), msh_tab.reshape(-1, 1),  msh_acc_tab.reshape(-1, 1), elipsh_tab.reshape(-1, 1), polarsh_tab.reshape(-1, 1), con_sh_tab.reshape(-1, 1), msat_tab.reshape(-1, 1), elipsat_tab.reshape(-1, 1), polarsat_tab.reshape(-1, 1), tb_sat_tab.reshape(-1, 1)))

#                     result_halogal_par.append(halogal_par_mat)
#                 # End: If at least one subhalo exists in loop-2 of host halos

#                 Mhosthl_tab_re[j] = mh-mshsat_tot

    Mcenl_ave = lens_gals.stellarmass_halomass(Mhosthl_tab_re / (cosmo_col.H0 / 100.), zl_tab, g.paramc, g.frac_SM_IMF) * (cosmo_col.H0 / 100.)
    Mcenl_scat = np.random.lognormal(0.0, g.sig_mcen, size=Mhosthl_tab_re.shape)
    Mcenl_tab = Mcenl_ave * Mcenl_scat

    elipcenl, polarcenl = lens_gals.set_gals_param(polarhl_tab)
    tb_cen = lens_gals.galaxy_size(Mhosthl_tab_re, Mcenl_tab/g.frac_SM_IMF, zl_tab, cosmo_col, model=g.TYPE_GAL_SIZE, scatter=True, sig_tb=g.sig_tb)
    halogal_par_mat = np.hstack((zl_tab.reshape(-1, 1), Mhosthl_tab_re.reshape(-1, 1), np.zeros_like(Mhosthl_tab).reshape(-1, 1), eliphl_tab.reshape(-1, 1), polarhl_tab.reshape(-1, 1), conhl_tab.reshape(-1, 1),
                                    Mcenl_tab.reshape(-1, 1), elipcenl.reshape(-1, 1), polarcenl.reshape(-1, 1), tb_cen.reshape(-1, 1)))

    halo_gal_pop_array = np.append(halo_gal_pop_array, halogal_par_mat, axis=0)
# data_frame_pop = pd.DataFrame(halo_gal_pop_list)

# data_frame_pop

In [99]:
columns_pop = ["z", "halo_mass", "halo_mass_acc", "e_h" , "p_h", "concentration", "stellar_mass", "e_g", "p_g", "tb"]
data_frame_pop = pd.DataFrame(halo_gal_pop_array)
data_frame_pop.columns =columns_pop
data_frame_pop

,z,halo_mass,halo_mass_acc,e_h,p_h,concentration,stellar_mass,e_g,p_g,tb
0,0.075,2.600160e+11,0.0,0.145123,115.552317,9.760382,5.865524e+09,0.373607,140.399393,0.365735
1,0.091,2.182730e+11,0.0,0.182705,-25.663141,13.494212,3.716412e+09,0.232750,-90.596291,0.397336
2,0.105,1.462177e+11,0.0,0.510557,-59.927349,9.158950,1.803778e+09,0.266069,-69.402165,0.689494
3,0.107,2.857591e+11,0.0,0.082263,-39.738060,13.614227,5.151813e+09,0.599937,33.779926,0.203443
4,0.113,1.606941e+11,0.0,0.294678,-26.069737,10.675590,2.071995e+09,0.169438,-8.782146,0.776613
...,...,...,...,...,...,...,...,...,...,...
32324,4.997,1.023293e+11,0.0,0.099154,111.671446,2.448956,6.121657e+08,0.297413,97.413709,0.184843
32325,4.997,1.037528e+11,0.0,0.096262,-45.937216,4.559088,3.953498e+08,0.468007,0.791494,0.041592
32326,4.998,1.078947e+11,0.0,0.125211,40.247689,4.835674,3.391498e+08,0.277973,-39.793874,0.045571
32327,4.998,1.227439e+11,0.0,0.542791,-155.775944,4.452229,5.875778e+08,0.386369,-165.288244,0.099346


In [ ]:
def halo_galaxy_population(sky_area,cosmo,cosmo_col,z_min,z_max,log10host_halo_mass_min,log10host_halo_mass_max,
                           log10subhalo_mass_min, sigma_host_halo_concentration, sigma_subhalo_concentration,
                           sigma_central_galaxy_mass, sigma_satellite_galaxy_mass, sig_tb,
                           TYPE_GAL_SIZE,frac_SM_IMF, TYPE_SMHM, switch_sub,**kwargs):
    dz = 0.001
    dlogMh = 0.001
    dlnMh = np.log(10**dlogMh)
    # cosmological parameters
    area = sky_area.value
    halo_gal_pop_array = np.array([])
    halo_gal_pop_array = np.empty((0,10), float)
    zz = np.arange(z_min, z_max + dz,dz)
    Mh_min = 10**log10host_halo_mass_min
    Mh_max = 10**log10host_halo_mass_max
    MMh = 10**np.arange(np.log10(Mh_min),
                        np.log10(Mh_max), dlogMh)
    min_Msh =10**log10subhalo_mass_min
    paramc, params = lens_gals.gals_init(TYPE_SMHM)
    sig_c = sigma_host_halo_concentration
    sig_csh = sigma_subhalo_concentration
    sig_mcen = sigma_central_galaxy_mass
    sig_msat = sigma_satellite_galaxy_mass

    for z in zz:
        zz2 = np.full(len(MMh), z)
        NNh = area * \
            lens_halo.dNhalodzdlnM_lens(MMh, zz2, cosmo_col) * dlnMh * dz
        Nh = np.random.poisson(NNh)
        indices = np.nonzero(Nh)[0]
        if len(indices) == 0:
            continue

        zl_tab = np.repeat(zz2[indices], Nh[indices])
        Mhosthl_tab  = np.repeat(MMh[indices], Nh[indices])
        conhl_tab = lens_halo.concent_m_w_scatter(Mhosthl_tab, z, sig_c)
        # in physical [Mpc/h]
        eliphl_tab, polarhl_tab = solve_lenseq.gene_e_ang_halo(Mhosthl_tab)

        mshsat_tot = 0
        Mhosthl_tab_re = Mhosthl_tab

        Mcenl_ave = lens_gals.stellarmass_halomass(Mhosthl_tab_re / (cosmo.H0 / 100.), zl_tab, paramc, frac_SM_IMF) * (cosmo.H0 / 100.)
        Mcenl_scat = np.random.lognormal(0.0, sig_mcen, size=Mhosthl_tab_re.shape)
        Mcenl_tab = Mcenl_ave * Mcenl_scat

        elipcenl, polarcenl = lens_gals.set_gals_param(polarhl_tab)
        tb_cen = lens_gals.galaxy_size(Mhosthl_tab_re, Mcenl_tab/frac_SM_IMF, zl_tab, cosmo_col, model=TYPE_GAL_SIZE, scatter=True, sig_tb=sig_tb)
        halogal_par_mat = np.hstack((zl_tab.reshape(-1, 1), Mhosthl_tab_re.reshape(-1, 1), np.zeros_like(Mhosthl_tab).reshape(-1, 1), eliphl_tab.reshape(-1, 1), polarhl_tab.reshape(-1, 1), conhl_tab.reshape(-1, 1),
                                        Mcenl_tab.reshape(-1, 1), elipcenl.reshape(-1, 1), polarcenl.reshape(-1, 1), tb_cen.reshape(-1, 1)))

        halo_gal_pop_array = np.append(halo_gal_pop_array, halogal_par_mat, axis=0)

        halo_gal_pop_array = np.append(halo_gal_pop_array, halogal_par_mat, axis=0)
    columns_pop = ["z", "halo_mass", "halo_mass_acc", "e_h" , "p_h", "concentration", "stellar_mass", "e_g", "p_g", "tb"]
    data_frame_pop = pd.DataFrame(halo_gal_pop_array)
    data_frame_pop.columns =columns_pop

    return data_frame_pop

In [3]:

import os
path = os.getcwd()
slhammocks_config = os.path.join(path, "../data/SL-Hammocks/gal_pop_Salpeter_10deg2_zl2.csv")

df = pd.read_csv(slhammocks_config)
# mlens_halo = np.where(df["mass_sh"]==-1, df["mass_hh"], df["mass_sh"])
# mlens_gal = np.where(df["mass_sh"]==-1, df["mass_cen"], df["mass_sat"])
# lens_con = np.where(df["mass_sh"]==-1, df["param1_hh"], df["param1_sh"])
# lens_tb = np.where(df["mass_sh"]==-1, df["param1_cen"], df["param1_sat"])
# elip_lens_gal = np.where(df["mass_sh"]==-1, df["elip_cen"], df["elip_sat"])
# df["mlens_halo"]=mlens_halo
# df["mlens_gal"]=mlens_gal
# df["lens_tb"]=lens_tb
# df["lens_con"]=lens_con
# df["elip_lens_gal"] = elip_lens_gal
# df = df.rename(columns={'zl_hh': 'z'})
df = df.rename(columns={'zl': 'z'})
df = df.rename(columns={'m_g': 'm_gal'})
df = df.rename(columns={'m_h': 'm_halo'})

data_area = 10.0 #deg2
if(sky_area.value>10.0):
    print("Now sky_area should be lower than 10. Now we set sky_area=10.0.")
thinp = int((data_area/sky_area).value)

In [ ]:
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from slsim.Pipelines.skypy_pipeline import SkyPyPipeline
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
import numpy as np
from slsim.selection import deflector_cut
import corner
from lenstronomy.Util import constants
from colossus.cosmology import cosmology
import numpy.random as random
from slsim.Util import param_util
import pandas as pd
import sys
from slsim.lens import Lens
from slsim.Deflectors.deflector import Deflector
# sys.path.append('../slsim')
# from lens_pop import LensPop
# from lensed_population_base import LensedPopulationBase
# sys.path.append('../slsim/Deflectors')  # 別ディレクトリのパスを追加
# from deflectors_base import DeflectorsBase
# from compound_lens_halos_galaxies import CompoundLensHalosGalaxies
# from lenstronomy.Util import constants
# from velocity_dispersion import vel_disp_composite_model
# sys.path.append('../slsim/Pipelines')  # 別ディレクトリのパスを追加
# from sl_hammocks_pipeline import SLHammocksPipeline


In [ ]:
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from slsim.Pipelines.skypy_pipeline import SkyPyPipeline
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
import numpy as np
from slsim.selection import deflector_cut
import corner
from lenstronomy.Util import constants
from colossus.cosmology import cosmology
import numpy.random as random
from slsim.Util import param_util
import pandas as pd
import sys
from slsim.lens import Lens
from slsim.Deflectors.deflector import Deflector
# sys.path.append('../slsim')
# from lens_pop import LensPop
# from lensed_population_base import LensedPopulationBase
# sys.path.append('../slsim/Deflectors')  # 別ディレクトリのパスを追加
# from deflectors_base import DeflectorsBase
# from compound_lens_halos_galaxies import CompoundLensHalosGalaxies
# from lenstronomy.Util import constants
# from velocity_dispersion import vel_disp_composite_model
# sys.path.append('../slsim/Pipelines')  # 別ディレクトリのパスを追加
# from sl_hammocks_pipeline import SLHammocksPipeline


In [4]:
df

,Unnamed: 0,m_halo,m_gal,z,vel_disp,tb,con,m_acc,elip_lens_gal
0,0,1.000000e+10,1.000000e+10,0.050000,150.000000,0.050000,2.000000,0.0,0.200000
1,1,1.668101e+10,1.668101e+10,0.266667,155.555556,0.055556,2.888889,0.0,0.244444
2,2,2.782559e+10,2.782559e+10,0.483333,161.111111,0.061111,3.777778,0.0,0.288889
3,3,4.641589e+10,4.641589e+10,0.700000,166.666667,0.066667,4.666667,0.0,0.333333
4,4,7.742637e+10,7.742637e+10,0.916667,172.222222,0.072222,5.555556,0.0,0.377778
5,5,1.291550e+11,1.291550e+11,1.133333,177.777778,0.077778,6.444444,0.0,0.422222
6,6,2.154435e+11,2.154435e+11,1.350000,183.333333,0.083333,7.333333,0.0,0.466667
7,7,3.593814e+11,3.593814e+11,1.566667,188.888889,0.088889,8.222222,0.0,0.511111
8,8,5.994843e+11,5.994843e+11,1.783333,194.444444,0.094444,9.111111,0.0,0.555556
9,9,1.000000e+12,1.000000e+12,2.000000,200.000000,0.100000,10.000000,0.0,0.600000


In [10]:
gg_lens_pop._lens_galaxies.draw_deflector()

Unnamed: 0       5.000000e+00
m_halo           1.291550e+11
m_gal            1.291550e+11
z                1.133333e+00
vel_disp         1.777778e+02
tb               7.777778e-02
con              6.444444e+00
m_acc            0.000000e+00
elip_lens_gal    4.222222e-01
mag_g           -1.000000e+00
mag_r           -1.000000e+00
mag_i           -1.000000e+00
mag_z           -1.000000e+00
mag_Y           -1.000000e+00
e1_light        -2.138122e-01
e2_light         5.771732e-02
e1_mass         -1.751155e-01
e2_mass          5.916205e-02
Name: 5, dtype: float64

In [5]:
# define a cosmology

cosmo_col = cosmology.setCosmology('planck18')
cosmo = FlatLambdaCDM(H0=cosmo_col.H0, Om0=cosmo_col.Om0)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml'
# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop2 = LensPop(
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    sky_area=sky_area,
    cosmo=cosmo,
)

In [ ]:
# define a cosmology

cosmo_col = cosmology.setCosmology('planck18')
cosmo = FlatLambdaCDM(H0=cosmo_col.H0, Om0=cosmo_col.Om0)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml'
# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    sky_area=sky_area,
    cosmo=cosmo,
)

In [12]:
filters=None
pipeline = SkyPyPipeline(
                skypy_config=skypy_config,
                sky_area=sky_area,
                filters=filters,
                cosmo=cosmo,
            )

In [17]:
gal_list = pipeline.red_galaxies
gal_list.sort("stellar_mass")
gal_list.reverse()
np.log10(min(gal_list["stellar_mass"])),np.log10(9055835950204)

(7.180317718856675, 12.956928546511325)

## Elliptical galaxies class

In [5]:
from slsim.Deflectors.elliptical_lens_galaxies import (
                EllipticalLensGalaxies)
kwargs_mass2light = {}
lens_galaxies = EllipticalLensGalaxies(
                pipeline.red_galaxies,
                kwargs_cut=kwargs_deflector_cut,
                kwargs_mass2light=kwargs_mass2light,
                cosmo=cosmo,
                sky_area=sky_area,
            )

In [2]:
import numpy.random as random
from slsim.Deflectors.elliptical_lens_galaxies import (
                elliptical_projected_eccentricity)
index = random.randint(0, lens_galaxies._num_select - 1)
deflec = lens_galaxies._galaxy_select[index]
a,b,c,d = elliptical_projected_eccentricity(**deflec)
print(deflec),print(a,b,c,d)

NameError: name 'lens_galaxies' is not defined

## Compound class

In [41]:
class CompoundLensHalosGalaxies(DeflectorsBase):
    """Class describing compound lens model based on the halo model."""

    def __init__(self, halo_galaxy_list, kwargs_cut, kwargs_mass2light, cosmo, sky_area):
        """

        :param halo_galaxy_list: list of dictionary with lens parameters of
            elliptical dark matte haloes and galaxies (currently supporting SL-Hammocks pipelines)
        :param kwargs_cut: cuts in parameters: band, band_mag, z_min, z_max
        :type kwargs_cut: dict
        # :param kwargs_mass2light: mass-to-light relation
        :param cosmo: astropy.cosmology instance
        :type sky_area: `~astropy.units.Quantity`
        :param sky_area: Sky area over which galaxies are sampled. Must be in units of
            solid angle.
        ## MEMO: DeflectorsBase's inputs are deflector_table, kwargs_cut, cosmo, sky_area
        """
        super().__init__(
            deflector_table=halo_galaxy_list,
            kwargs_cut=kwargs_cut,
            cosmo=cosmo,
            sky_area=sky_area,
        )

        n = len(halo_galaxy_list)
        column_names = halo_galaxy_list.columns
        if "vel_disp" not in column_names:
            halo_galaxy_list["vel_disp"] = -np.ones(n)
        if "mag_g" not in column_names:
            halo_galaxy_list["mag_g"] = -np.ones(n)
        if "mag_r" not in column_names:
            halo_galaxy_list["mag_r"] = -np.ones(n)
        if "mag_i" not in column_names:
            halo_galaxy_list["mag_i"] = -np.ones(n)
        if "mag_z" not in column_names:
            halo_galaxy_list["mag_z"] = -np.ones(n)
        if "mag_Y" not in column_names:
            halo_galaxy_list["mag_Y"] = -np.ones(n)
        if "e1_light" not in column_names or "e2_light" not in column_names:
            halo_galaxy_list["e1_light"] = -np.ones(n)
            halo_galaxy_list["e2_light"] = -np.ones(n)
        if "e1_mass" not in column_names or "e2_mass" not in column_names:
            halo_galaxy_list["e1_mass"] = -np.ones(n)
            halo_galaxy_list["e2_mass"] = -np.ones(n)
        # TODO: check

        # num_total = len(halo_galaxy_list)
        # z_min, z_max = 0, np.max(halo_galaxy_list["z"])

        self._galaxy_select = deflector_cut(halo_galaxy_list, **kwargs_cut)
        # Currently only supporting redshift cut
        self._num_select = len(self._galaxy_select)
        self._cosmo = cosmo

        # TODO: random reshuffle of matched list

    def draw_deflector(self):
        """
        :return: dictionary of complete parameterization of deflector
        """
        cosmo = self._cosmo
        index = random.randint(0, self._num_select - 1)
        deflector_ori = self._galaxy_select.iloc[index]
        deflector = deflector_ori.copy()
        # if deflector["vel_disp"] == -1:
        theta_eff = deflector['tb']/0.551 # [arcsec]
        reff = (theta_eff*cosmo.angular_diameter_distance(deflector['z'])*constants.arcsec).value # physical Mpc
        hubble = cosmo.H0.value/100.
        vel_disp = vel_disp_composite_model(
                theta_eff, deflector['m_gal']/(hubble), reff, max(deflector['m_halo'],deflector['m_acc'])/(hubble),deflector['con'], cosmo, deflector['z'])
        print(vel_disp)
            # vel_disp = vel_disp_composite_model(theta_eff, deflector['mlens_gal']/(hubble), reff, deflector['mlens_halo']/(hubble),deflector['lens_con'], deflector['z'], deflector['zs'], cosmo)
            # deflector["vel_disp"] = vel_disp
        if deflector["mag_g"] == -1 or deflector["mag_r"] or  deflector["mag_i"] == -1 or deflector["mag_z"] or deflector["mag_Y"] == -1:
            mag_g, mag_r, mag_i, mag_z, mag_Y = 0,0,0,0,0 # TODO: make function if needed
        if deflector["e1_light"] == -1 or deflector["e2_light"] == -1:
            e1_light, e2_light, e1_mass, e2_mass = elliptical_projected_eccentricity_galaxy(
                **deflector
            ) # TODO: make function if needed
            deflector["e1_light"] = e1_light
            deflector["e2_light"] = e2_light
            deflector["e1_mass"] = e1_mass
            deflector["e2_mass"] = e2_mass
        return deflector

    def deflector_number(self):
        """

        :return: number of deflectors
        """
        number = self._num_select
        return number


def elliptical_projected_eccentricity_galaxy(elip_lens_gal, **kwargs): #TODO: add row of elip_lens_gal if needed
    """Projected eccentricity of elliptical galaxies as a function of other deflector
    parameters.

    :param ellipticity: eccentricity amplitude
    :type ellipticity: float [0,1)
    :param kwargs: deflector properties
    :type kwargs: dict
    :return: e1_light, e2_light,e1_mass, e2_mass eccentricity components
    """
    ellipticity = elip_lens_gal
    e_light = param_util.epsilon2e(ellipticity)
    phi_light = np.random.uniform(0, np.pi)
    e1_light = e_light * np.cos(phi_light)
    e2_light = e_light * np.sin(phi_light)
    e_mass = 0.5 * ellipticity + np.random.normal(loc=0, scale=0.1)
    phi_mass = phi_light + np.random.normal(loc=0, scale=0.1)
    e1_mass = e_mass * np.cos(phi_mass)
    e2_mass = e_mass * np.sin(phi_mass)
    return e1_light, e2_light, e1_mass, e2_mass


def vel_disp_composite_model2(r, m_star, rs_star, m_halo, c_halo, cosmo, z_lens):
    """Computes the luminosity weighted velocity dispersion for a deflector with a
    stellar Hernquist profile and a NFW halo profile, assuming isotropic anisotropy.

    :param r: radius of the luminosity-weighted velocity dispersion [arcsec]
    :param m_star: stellar mass [M_sun]
    :param rs_star: stellar half light radius [physical Mpc]
    :param m_halo: Halo mass [physical M_sun]
    :param c_halo: halo concentration
    :param cosmo: cosmology
    :type cosmo: ~astropy.cosmology class
    :param z_lens: redshift of the deflector
    :return: velocity dispersion [km/s]
    """
    kwargs_model = {
        "mass_profile_list": ["HERNQUIST", "NFW"],
        "light_profile_list": ["HERNQUIST"],
        "anisotropy_model": "const",
    }

    # turn physical masses to lenstronomy units
    from lenstronomy.Cosmo.lens_cosmo import LensCosmo

    lens_cosmo = LensCosmo(z_lens=z_lens, z_source=z_lens+0.5, cosmo=cosmo)
    # Hernquist profile
    sigma0, rs_angle_hernquist = lens_cosmo.hernquist_phys2angular(
        mass=m_star, rs=rs_star
    )

    rs_angle_nfw, alpha_Rs = lens_cosmo.nfw_physical2angle(M=m_halo, c=c_halo)
    kwargs_mass = [
        {"sigma0": sigma0, "Rs": rs_angle_hernquist, "center_x": 0, "center_y": 0},
        {"alpha_Rs": alpha_Rs, "Rs": rs_angle_nfw, "center_x": 0, "center_y": 0},
    ]
    kwargs_light = [{"amp": 1, "Rs": rs_angle_hernquist, "center_x": 0, "center_y": 0}]
    kwargs_anisotropy = {"beta": 0}

    from lenstronomy.GalKin.numeric_kinematics import NumericKinematics

    kwargs_numerics = {
        "interpol_grid_num": 1000,
        "log_integration": True,
        "max_integrate": 100,
        "min_integrate": 0.0001,
        "max_light_draw": None,
        "lum_weight_int_method": True,
    }
    kwargs_cosmo = {"d_d": lens_cosmo.dd, "d_s": lens_cosmo.ds, "d_ds": lens_cosmo.dds}


    num_kin = NumericKinematics(kwargs_model, kwargs_cosmo, **kwargs_numerics)
    vel_disp = num_kin.lum_weighted_vel_disp(
        r, kwargs_mass, kwargs_light, kwargs_anisotropy
    )
    return vel_disp



In [4]:
import pandas as pd
# columns = [
#     'nim', 'mori', 'mobs', 'sep', 'ein', 'fs', 'frac_sh_trunc', 'mmin', 'flag_out', 'zs', 'xs', 'ys'
# ]

# for i in range(5):
#     suffix = "_sh" if i == 0 else "_sat" if i == 1 else "_hh" if i == 2 else "_cen" if i == 3 else "_pert"
#     columns.extend([
#         f'm_type{suffix}', f'zl{suffix}', f'mass{suffix}', f'xl{suffix}',
#         f'yl{suffix}', f'elip{suffix}', f'pol{suffix}', f'param1{suffix}'
#     ])
# file_path = '/Users/ktabe/Research/strong-lens-SNe/genmock_halo/result/default_kapst/'
# prefix = 'default_LSST_kapst'
# data = np.loadtxt(file_path+"/"+prefix+"_result.dat")
# df = pd.DataFrame(data, columns=columns)
# lens_class = df
# lens_class
df = pd.read_csv("/Users/ktabe/Research/strong-lens-SNe/slsim_KTA/data/SL-Hammocks/qso_salpeter_LSST_multi.csv")

In [17]:
import os
import slsim
path = os.getcwd()
module_path, _ = os.path.split(path)
slhammocks_config = os.path.join(path, "../data/SL-Hammocks/qso_salpeter_LSST_multi.csv")
df = pd.read_csv(slhammocks_config)

In [18]:
mlens_halo = np.where(df["mass_sh"]==-1, df["mass_hh"], df["mass_sh"])
mlens_gal = np.where(df["mass_sh"]==-1, df["mass_cen"], df["mass_sat"])
lens_con = np.where(df["mass_sh"]==-1, df["param1_hh"], df["param1_sh"])
lens_tb = np.where(df["mass_sh"]==-1, df["param1_cen"], df["param1_sat"])
elip_lens_gal = np.where(df["mass_sh"]==-1, df["elip_cen"], df["elip_sat"])
df["mlens_halo"]=mlens_halo
df["mlens_gal"]=mlens_gal
df["lens_tb"]=lens_tb
df["lens_con"]=lens_con
df["elip_lens_gal"] = elip_lens_gal
df = df.rename(columns={'zl_hh': 'z'})

In [2]:
sys.path.append('../slsim/Pipelines')  # 別ディレクトリのパスを追加
from sl_hammocks_pipeline import SLHammocksPipeline

In [11]:
# define a cosmology

cosmo_col = cosmology.setCosmology('planck18')
cosmo = FlatLambdaCDM(H0=cosmo_col.H0, Om0=cosmo_col.Om0)
sky_area = Quantity(value=0.1, unit="deg2")
kwargs_deflector_cut = {"z_min": 0.01, "z_max": 2.5}

In [6]:
pipeline2 = SLHammocksPipeline(
                slhammocks_config=None,
                sky_area=sky_area,
                cosmo=cosmo,
            )

In [20]:
n = len(halo_galaxy_list)
column_names = halo_galaxy_list.columns
if "vel_disp" not in column_names:
    halo_galaxy_list["vel_disp"] = -np.ones(n)
else:
    print("include")

include


In [23]:
kwargs_deflector_cut = {"z_min": 0.01, "z_max": 2.5}
galaxy_select = deflector_cut(halo_galaxy_list, **kwargs_deflector_cut)
galaxy_select

NameError: name 'halo_galaxy_list' is not defined

### Test vel_disp_compound

In [45]:
pipeline2._pipeline.columns

Index(['Unnamed: 0', 'index', 'z', 'm_halo', 'm_acc', 'e_h', 'p_h', 'con',
       'm_gal', 'e_g', 'p_g', 'tb', 'vel_disp'],
      dtype='object')

In [24]:
import numpy.random as random
import sys

kwargs_mass2light = {}
lens_galaxies = CompoundLensHalosGalaxies(
                pipeline2._pipeline.copy(),
                kwargs_cut=kwargs_deflector_cut,
                kwargs_mass2light=kwargs_mass2light,
                cosmo=cosmo,
                sky_area=sky_area,
            )

In [25]:
lens_galaxies._galaxy_select

,Unnamed: 0,index,z,m_halo,m_acc,e_h,p_h,con,m_gal,e_g,...,vel_disp,mag_g,mag_r,mag_i,mag_z,mag_Y,e1_light,e2_light,e1_mass,e2_mass
0,0,1.0,0.019,1.244515e+12,0.000000e+00,0.187930,-144.374000,8.977181,4.517617e+10,0.298916,...,128.801900,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
100,100,101.0,0.144,1.339274e+12,0.000000e+00,0.247963,112.869200,11.008350,4.194567e+10,0.025451,...,115.253096,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
200,200,201.0,0.167,3.723917e+11,0.000000e+00,0.271244,-9.635914,8.594441,8.539675e+09,0.429663,...,70.951111,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
300,300,301.0,0.184,2.403459e+11,7.791242e+11,0.322615,-111.397300,31.934680,4.313094e+10,0.431950,...,173.523664,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
400,400,401.0,0.213,4.813280e+10,1.539985e+11,0.293230,-98.551830,22.060570,3.007909e+09,0.021218,...,70.217042,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836100,836100,836101.0,1.990,1.320623e+12,0.000000e+00,0.361820,-125.420200,1.948017,3.128009e+10,0.319185,...,136.727897,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
836200,836200,836201.0,1.993,4.675144e+10,1.679422e+11,0.151623,113.741200,12.795430,1.465869e+09,0.424344,...,92.630333,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
836300,836300,836301.0,1.995,4.027170e+11,0.000000e+00,0.239865,21.876960,3.597686,3.384027e+09,0.265425,...,77.866145,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
836400,836400,836401.0,1.998,4.989141e+10,1.814349e+11,0.584665,-31.060090,9.095567,1.346018e+09,0.349045,...,60.557533,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [52]:
# test = lens_galaxies.draw_deflector()
i=120
deflector = lens_galaxies._galaxy_select.iloc[i]

theta_eff = deflector['tb']/0.551 # [arcsec]
reff = (theta_eff*cosmo.angular_diameter_distance(deflector['z'])*constants.arcsec).value # physical Mpc
hubble = cosmo.H0.value/100.

vel_disp = vel_disp_composite_model(theta_eff, deflector['m_gal']/(hubble), reff, max(deflector['m_halo'],deflector['m_acc'])/(hubble), deflector['con'], cosmo, deflector['z'])
vel_disp2 = vel_disp_composite_model2(theta_eff, deflector['m_gal']/(hubble), reff, deflector['m_halo']/(hubble), deflector['con'], cosmo, deflector['z'])

# re_pop_sub = (deflector['tb']*theta_eff*cosmo.angular_diameter_distance(deflector['z'])*constants.arcsec).value

# vel_disp_composite_model(deflector['tb']/0.551, deflector['m_g'].iloc[i]/(cosmo.H0/100.),
#                                 re_pop_sub, max(df_sub['m_h'].iloc[i],df_sub['m_acc'].iloc[i])/(cosmo.H0/100.),
#                                 df_sub['con'].iloc[i], df_sub['zl'].iloc[i], max(1.5,df_sub['zl'].iloc[i]+0.5),cosmop18))
#     vel_pop_sub_ar = np.array(vel_pop_sub)
print(deflector['m_halo'],deflector['m_acc'],vel_disp,vel_disp2, lens_galaxies._galaxy_select.iloc[i]["vel_disp"],lens_galaxies._galaxy_select.iloc[i],cosmo.angular_diameter_distance(deflector['z']))
# # # test

796159400000.0 0.0 128.4151557557737 128.4125275148259 128.39161113322533 Unnamed: 0    1.200000e+04
index         1.200100e+04
z             7.420000e-01
m_halo        7.961594e+11
m_acc         0.000000e+00
e_h           3.716677e-01
p_h           1.407213e+02
con           1.135522e+01
m_gal         2.634698e+10
e_g           3.702175e-01
p_g           8.901383e+01
tb            8.547288e-02
vel_disp      1.283916e+02
mag_g        -1.000000e+00
mag_r        -1.000000e+00
mag_i        -1.000000e+00
mag_z        -1.000000e+00
mag_Y        -1.000000e+00
e1_light     -1.000000e+00
e2_light     -1.000000e+00
e1_mass      -1.000000e+00
e2_mass      -1.000000e+00
Name: 12000, dtype: float64 1551.5006031719888 Mpc


In [46]:
hubble

0.6766

0.0897516624218156

In [147]:
index = random.randint(0, lens_galaxies._num_select - 1)
deflec = lens_galaxies._galaxy_select[index]
a,b,c,d = elliptical_projected_eccentricity(**deflec)
print(deflec),print(a,b,c,d)

lens_class.(by='', ascending=True, inplace=True)

In [ ]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_type="halomodel",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml',
    sky_area=sky_area,
    cosmo=cosmo,
)

In [4]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens = Lens(
    source_dict,
    deflector_dict,
    cosmo,
    deflector_type="EPL",
    source_type="extended",
    lens_equation_solver="lenstronomy_analytical",
    variability_model=None,
    kwargs_variability=None,
    sn_type=None,
    sn_absolute_mag_band=None,
    sn_absolute_zpsys=None,
    test_area=4 * np.pi,
    mixgauss_means=None,
    mixgauss_stds=None,
    mixgauss_weights=None,
    magnification_limit=0.01,
    light_profile="single_sersic",
    lightcurve_time=None,
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config='/Users/ktabe/Research/strong-lens-SNe/slsim/data/SkyPy/lsst-like.yml',
    sky_area=sky_area,
    cosmo=cosmo,
)

TypeError: Lens.__init__() got an unexpected keyword argument 'deflector_type'

## Generate images of random lenses
The LensingPlots() class has the functionality to draw random lenses and makes an image of it.
Currently

default settings in lenstronomy are chosen for the LSST image settings. These will be able to be replaced with the

LSST simulation tools.

In [ ]:
# make some cuts in the image separations and limited magnitudes of the arc
kwargs_lens_cut_plot = {
    "min_image_separation": 0.8,
    "max_image_separation": 10,
    "mag_arc_limit": {"g": 23, "r": 23, "i": 23},
}


gg_plot = LensingPlots(gg_lens_pop, num_pix=64, coadd_years=10)

# generate montage indicating which bands are used for the rgb color image
fig, axes = gg_plot.plot_montage(
    rgb_band_list=["i", "r", "g"],
    add_noise=True,
    n_horizont=5,
    n_vertical=2,
    kwargs_lens_cut=kwargs_lens_cut_plot,
)
plt.show()

## Generate the full population
We are using the instance of the LensPop() class to draw the full population within specified cuts in a Monte Carlo process.

In [ ]:
# specifying cuts of the population
kwargs_lens_cuts = {"mag_arc_limit": {"g": 28}}
# drawing population
gg_lens_population = gg_lens_pop.draw_population(kwargs_lens_cuts=kwargs_lens_cuts)

## Represent key quantities of full population in corner plots
We calculate few key quantities of the lenses. The full population is represented each with a Lens() class

object that allows to compute and return these (and more) quantities.

In [ ]:
print("Number of lenses:", len(gg_lens_population))

lens_samples = []
labels = [
    r"$\sigma_v$",
    r"$\log(M_{*})$",
    r"$\theta_E$",
    r"$z_{\rm l}$",
    r"$z_{\rm s}$",
    r"$m_{\rm source}$",
    r"$m_{\rm lens}$",
]

for gg_lens in gg_lens_population:
    vel_disp = gg_lens.deflector_velocity_dispersion()
    m_star = gg_lens.deflector_stellar_mass()
    theta_e = gg_lens.einstein_radius
    zl = gg_lens.deflector_redshift
    zs = gg_lens.source_redshift
    source_mag = gg_lens.extended_source_magnitude(band="g", lensed=True)
    deflector_mag = gg_lens.deflector_magnitude(band="g")
    lens_samples.append(
        [vel_disp, np.log10(m_star), theta_e, zl, zs, source_mag, deflector_mag]
    )

In [ ]:
hist2dkwargs = {
    "plot_density": False,
    "plot_contours": False,
    "plot_datapoints": True,
    "color": "b",
    "data_kwargs": {"ms": 5},
}
corner.corner(np.array(lens_samples), labels=labels, **hist2dkwargs)
plt.show()